In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
model = YOLO('models/yolov8x-pose-p6.pt') 

# Archive 1

In [3]:
def _convert_result_to_string(result):
    output = '['
    for i, r in enumerate(result):
        if i==20: break
        output += r.tojson() + ','
    output = output[:-1]+']'
    return eval(output), r.orig_shape

In [4]:
def _extract_keypoints(data, orig_shape):
  d={}

  keypoints_batch =[]
  batch_size=20
  max_persons = 10
  for i, frame in enumerate(data):
    track_id_keypoints = []
    for person_id in range(max_persons):
      
      if person_id>=len(frame):
        track_id_keypoints = d.get(person_id, [])
        track_id_keypoints.append(np.zeros(34))
        d[person_id] = track_id_keypoints

      else:
          keypoints = frame[person_id]["keypoints"]
          x = np.array(keypoints["x"]) / orig_shape[1]
          y = np.array(keypoints["y"]) / orig_shape[0]
          XY = np.asarray([j for j in zip(x, y)]).reshape(34)
      
          track_id_keypoints = d.get(person_id, [])
          track_id_keypoints.append(XY)
          d[person_id] = track_id_keypoints

  return np.asarray(list(d.values()))



In [5]:

# image_path = 'examples/videos/V12.mp4'
# result = model.track(image_path, device='0', show=True, verbose=False, stream=True)
# cv2.destroyAllWindows()

# data, orig_shape = _convert_result_to_string(result)

# keypoints_batch = _extract_keypoints(data, orig_shape)

# keypoints_batch.shape
# cv2.destroyAllWindows()

In [12]:
# load model
from keras.models import load_model
fusion_model = load_model('models/detector.keras')

In [13]:
fusion_model

In [14]:
from keras.models import load_model
def predictor(data):
    return fusion_model.predict(data, verbose=False)


In [19]:
import cv2
model = YOLO('models/yolov8n-pose.pt') 
cap = cv2.VideoCapture('examples/videos/Walking.mp4')

buffer_size = fusion_model.input_shape[1]
keypoints_buffer = np.zeros((10,1,34))
while True:
    ret, frame = cap.read()
    
    if ret == False:
        cv2.destroyAllWindows()
        cap.release()
        print("Error reading frame!")
        break

    height, width, channels = frame.shape
    if keypoints_buffer.shape[1] < buffer_size:
        result = model.track(frame, persist=True, verbose=False, device='0')
        data, orig_shape = _convert_result_to_string(result)
        keypoints_batch = _extract_keypoints(data, orig_shape)
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)
        # print(keypoints_buffer.shape)

    else:
        result = model.track(frame, persist=True, verbose=False, device='0')
        data, orig_shape = _convert_result_to_string(result)
        keypoints_batch = _extract_keypoints(data, orig_shape)

        if keypoints_buffer.shape[1] + keypoints_batch.shape[1] > buffer_size:
            excess_size = keypoints_buffer.shape[1] + keypoints_batch.shape[1] - buffer_size
            keypoints_buffer = keypoints_buffer[:, excess_size:, :]

        # Concatenate the new keypoints_batch to the circular buffer
        keypoints_buffer = np.concatenate((keypoints_buffer, keypoints_batch), axis=1)
        
        prediction = predictor(keypoints_buffer)
        
        for id, person in enumerate(data[0]):
            if prediction[id] > 0.5:
                x1 = int(person['box']['x1'])
                y1 = int(person['box']['y1'])
                x2 = int(person['box']['x2'])
                y2 = int(person['box']['y2'])
                print([(x1,y1), (x2,y2), prediction[id]], end='\r')
                frame = cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 5)
    
    if cv2.waitKey(1)==27:
        cv2.destroyAllWindows()
        cap.release()
        break


    cv2.imshow('Output',frame)
    
  

In [15]:
cv2.destroyAllWindows()
cap.release()